In [3]:
import json

source = r"D:\0Business\ML\WOS\Nature\databank\CrawlerData\2022\2022.json"
# 使用了2022年化学文献信息库
source = json.loads(open(source, "r", encoding="utf8").read())
source_ = source[0]
len(source)

22899

In [17]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np

def EmbedSentences(sentences):
    # Load the Universal Sentence Encoder module
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    model = hub.load(module_url)
    # Generate embeddings for the sentences
    embeddings = model(sentences)
    return embeddings

def eager_tensor_to_list(eager_tensor):
    tensor_as_numpy = eager_tensor.numpy()
    tensor_as_list = tensor_as_numpy.tolist()
    return tensor_as_list

def MetaData(source):
    source_ = {
        "identifier": source["identifier"],
        "title": source["title"],
        "abstract": source["abstract"],
        "publicationDate": source["publicationDate"],
        "publicationName": source["publicationName"],
        "openaccess": source["openaccess"],
        "publisher": source["publisher"],
        "publicationType": source["publicationType"],
        "genre": source["genre"],
        "doi": source["doi"]
    }
    return source_

In [11]:
id = 0
idList = []
metaList = []
vectorList = []
batch = 2000
while id < 200:
    print("Processing batch starting from ", id)
    source_ = source[id : id+batch]
    paragraphs = []
    for item in source_:
        try:
            text = item["title"] + ". " + item["abstract"]
            paragraphs.append(text)

            idList.append(item["doi"])

            metaInfo = MetaData(item)
            metaList.append(metaInfo)
        except Exception as e:
            print(">>> This item went wrong")
            print(item) 
            print(e)

    print("Processing batch starting from ", id)
    embeddings = EmbedSentences(paragraphs)
    embeddings_ = [eager_tensor_to_list(item) for item in embeddings]
    vectorList = vectorList + embeddings_

    id += batch


Processing batch starting from  0
Processing batch starting from  0


In [18]:
vectors = []
for i in range(len(idList)):
    data = {
        "id": idList[i],
        "metadata": MetaData(metaList[i]),
        "values": vectorList[i]
    }
    vectors.append(data)

In [23]:
namespace = "chemistry_journals"
logData = {
    "vectors": vectors[500: 1000],
    "namespace": namespace
}
log = open("log1-1000.json", "w", encoding="utf8")
log.write(json.dumps(logData, indent=4))
log.flush()